<p><img src="https://docs.oracle.com/en-us/iaas/Content/anomaly/images/mset2.png" width="300" align = "left"></p>
<p><img src="https://www.adaption-it.nl/wp-content/uploads/2019/02/Oracle-Logo.png.png" width="300" align = "right"></p>


# **<h1 align ="middle"><b> OCI Anomaly Detection </b></h1>**

# **Steps**

- Import libraries + authenticate using Config
- Load new data set and convert to JSON payload
- Send JSON payload and receive response

# **1. Libraries and authentication**

In [3]:
import oci
import time
import json
import pandas as pd
from datetime import datetime, date

from oci.config import from_file
from oci.ai_anomaly_detection.models import *
from oci.ai_anomaly_detection.anomaly_detection_client import AnomalyDetectionClient

from oci.ai_anomaly_detection.models.create_project_details import CreateProjectDetails
from oci.ai_anomaly_detection.models.create_data_asset_details import CreateDataAssetDetails
from oci.ai_anomaly_detection.models.data_source_details import DataSourceDetails
from oci.ai_anomaly_detection.models.data_source_details_object_storage import DataSourceDetailsObjectStorage

from oci.ai_anomaly_detection.models.create_model_details import CreateModelDetails
from oci.ai_anomaly_detection.models.model_training_details import ModelTrainingDetails

from oci.ai_anomaly_detection.models.data_item import DataItem
from oci.ai_anomaly_detection.models.inline_detect_anomalies_request import InlineDetectAnomaliesRequest

## authentication
config = from_file("/home/datascience/.oci/config")
SERVICE_ENDPOINT="https://anomalydetection.aiservice.eu-frankfurt-1.oci.oraclecloud.com"
ad_client = AnomalyDetectionClient(config, service_endpoint=SERVICE_ENDPOINT)

# **2. Load data and create payload**

In [4]:
#Load the data from a csv file with first column as timestamp
df = pd.read_csv("/home/datascience/xxx/xxxxxxx.csv")
signalNames = [e for e in df.columns if e != 'timestamp']

#Create the Payload from the dataframe
payloadData = []
for index, row in df.iterrows():
    timestamp = datetime.strptime(row['timestamp'], "%Y-%m-%dT%H:%M:%SZ")
    values = list(row[signalNames])
    dItem = DataItem(timestamp=timestamp, values=values)
    payloadData.append(dItem)

#create a subset
payloadData_short = payloadData[0:10]

# **3. Send payload to pre-built model and get response**

In [5]:
## model details
model_id = "[MODEL_OCID]"

## actual request
inline = InlineDetectAnomaliesRequest( model_id=model_id, request_type="INLINE", signal_names=signalNames, data=payloadData_short)
detect_res = ad_client.detect_anomalies(detect_anomalies_details=inline)

print("----DETECTING----")
print(detect_res.data)

----DETECTING----
{
  "detection_results": [
    {
      "anomalies": [
        {
          "actual_value": 10.0224,
          "anomaly_score": 0.6,
          "estimated_value": 1.1503004343005476,
          "imputed_value": null,
          "signal_name": "pressure_5"
        }
      ],
      "row_index": null,
      "score": 0.15247896857228543,
      "timestamp": "2019-01-01T00:00:04+00:00"
    },
    {
      "anomalies": [
        {
          "actual_value": 12.3838,
          "anomaly_score": 0.6,
          "estimated_value": 0.6492285168789096,
          "imputed_value": null,
          "signal_name": "temperature_2"
        }
      ],
      "row_index": null,
      "score": 0.1454964781633803,
      "timestamp": "2019-01-01T00:04:04+00:00"
    }
  ]
}
